# Prerequisite

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import pandas as pd
import numpy as np
from scipy.stats import variation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Process data from Lion

In [6]:
# for plotting the graph inside the notebook itself
# %matplotlib inline

# for plotting the graph interactively (outside)
# %matplotlib qt

# for plotting the graph interactively (inside)
%matplotlib widget

import matplotlib.pyplot as plt
import more_itertools as mit
import glob2

AXIS_INDEX = {
    'x': 'd1',
    'y': 'd2',
    'z': 'd3',
    'amp': 'amp'
}

PLOT_COLOR = ['b', 'r']
ZKSENSORS = ['UAC', 'GYR']

TIME_BEFORE = 50
TIME_AFTER = 250
TIME_DURATION = 150
MINIMAL_POINTS = 5
DIGITS = 6

DRAG_DURATION = 400
DOWNSAMPLE_RATE = '4ms'
# DOWNSAMPLE_RATE = None

COLUMNS = ['timestamp', 'sensor', 'touch', 'keyboard', 'orientation', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6']

amp = lambda df: np.sqrt(df['d1']**2 + df['d2']**2 + df['d3']**2)

def read_csv(file_path, platform='Android'):
    df = pd.read_csv(file_path, header=None, names=COLUMNS)
    df.loc[df['sensor'] == 'GYR', ['d1', 'd2', 'd3']] = df[df['sensor'] == 'GYR'][['d1', 'd2', 'd3']] - df[df['sensor'] == 'GYR'][['d4', 'd5', 'd6']].fillna(0).values
    df = df.drop(columns=['d4', 'd5', 'd6'])
    df[['d1', 'd2', 'd3']] = round(df[['d1', 'd2', 'd3']] * 10**DIGITS)
    
    return df
    
def plot_sensor_axis(name, axis, df, ptype='line', tdf=None):
    sensor_df = df.loc[df.iloc[:, 1] == name]
    sensor_df = sensor_df.reset_index(drop=True)
    sensor_df['timestamp'] = sensor_df['timestamp'] - sensor_df.iloc[0, 0]
    
    plt.figure()
    
    if ptype == 'line':
        touch_df = sensor_df.copy()
        non_touch_df = sensor_df.copy()
        
        touch_df[touch_df['touch'] == 0] = np.nan
        non_touch_df[touch_df['touch'] == 1] = np.nan
        
        plt.plot(touch_df['timestamp'], touch_df[AXIS_INDEX[axis]], c='r')
        plt.plot(non_touch_df['timestamp'], non_touch_df[AXIS_INDEX[axis]], c='b')
        
        if 'None' not in str(type(tdf)):
#             tdf['timestamp'] = tdf['timestamp'] - sensor_df.iloc[0, 0]
            plt.plot(tdf['timestamp'], tdf['touch'], c='g')
    else:
        plt.scatter(sensor_df['timestamp'], sensor_df[AXIS_INDEX[axis]], c=sensor_df[2])

    plt.xlabel('Timestamp')
    plt.ylabel("{} ({})".format(name, axis))
    
    plt.show()

    
def plot_for_slides(df, dev=False):
    
    plt.figure()
    axes = ['d3', 'd2']
    names = ['z', 'y']
    
    for idx, name in enumerate(['UAC', 'GYR']):
        plt.subplot(2, 1, idx + 1)
        
        sensor_df = df.loc[df.iloc[:, 1] == name]
        sensor_df = sensor_df.reset_index(drop=True)
        sensor_df['timestamp'] = sensor_df['timestamp'] - sensor_df.iloc[0, 0]
        
        touch_df = sensor_df.copy()
        non_touch_df = sensor_df.copy()

        touch_df[touch_df['touch'] == 0] = np.nan
        non_touch_df[touch_df['touch'] == 1] = np.nan


        if dev:
            plt.plot(touch_df['timestamp'][:-1], (touch_df[1:][axes[idx]].values - touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='r')
            plt.plot(non_touch_df['timestamp'][:-1], (non_touch_df[1:][axes[idx]].values - non_touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='b')
        else:
            plt.plot(touch_df['timestamp'], touch_df[axes[idx]] / (10 ** DIGITS), c='r')
            plt.plot(non_touch_df['timestamp'], non_touch_df[axes[idx]] / (10 ** DIGITS), c='b')
       
        plt.xlabel('Timestamp')
        plt.ylabel("{} ({})".format(name, names[idx]))
    
    plt.show()
    

def plot_for_slides_diff():
    df_swing = read_csv("data/Bot/data/Stan-Swing/android_build-Pixel_3-1565690457215.csv")
    df_desk = read_csv("data/Android/Processed/Stan-Desk/files/android_build-Pixel_3-1564483096188.csv")
    
    seg_swing = get_click_segs(df_swing, '5ms')[3][0]
    seg_desk = get_click_segs(df_desk, '5ms')[1][0]
    
    plt.figure()
    axes = ['d3', 'd2']
    names = ['z', 'y']
    
#     df = seg_desk
    
#     for idx, name in enumerate(['UAC', 'GYR']):
#         plt.subplot(2, 2, idx * 2 + 1)
        
#         sensor_df = df.loc[df.iloc[:, 1] == name]
#         sensor_df = sensor_df.reset_index(drop=True)
#         sensor_df['timestamp'] = sensor_df['timestamp'] - sensor_df.iloc[0, 0]
        
#         touch_df = sensor_df.copy()
#         non_touch_df = sensor_df.copy()

#         touch_df[touch_df['touch'] == 0] = np.nan
#         non_touch_df[touch_df['touch'] == 1] = np.nan

#         plt.plot(touch_df['timestamp'][:-1], (touch_df[1:][axes[idx]].values - touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='r')
#         plt.plot(non_touch_df['timestamp'][:-1], (non_touch_df[1:][axes[idx]].values - non_touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='b')
       
#         plt.xlabel('Timestamp')
#         plt.ylabel("{} ({})".format(name, names[idx]))
    
    df = seg_swing
    
    for idx, name in enumerate(['UAC', 'GYR']):
        plt.subplot(2, 2, (idx + 1) * 2)
        
        sensor_df = df.loc[df.iloc[:, 1] == name]
        sensor_df = sensor_df.reset_index(drop=True)
        sensor_df['timestamp'] = sensor_df['timestamp'] - sensor_df.iloc[0, 0]
        
        touch_df = sensor_df.copy()
        non_touch_df = sensor_df.copy()

        touch_df[touch_df['touch'] == 0] = np.nan
        non_touch_df[touch_df['touch'] == 1] = np.nan

        plt.plot(touch_df['timestamp'][:-1], (touch_df[1:][axes[idx]].values - touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='r')
        plt.plot(non_touch_df['timestamp'][:-1], (non_touch_df[1:][axes[idx]].values - non_touch_df[:-1][axes[idx]].values) / (10 ** DIGITS), c='b')
       
        plt.xlabel('Timestamp')
        plt.ylabel("{} ({})".format(name, names[idx]))
    
    plt.show()
    
    
def plot_sensor_amplitude(name, df, ptype='line'):
    sensor_df = df.loc[df.iloc[:, 1] == name]
    sensor_df = sensor_df.reset_index(drop=True)
    sensor_df['timestamp'] = sensor_df['timestamp'] - sensor_df.iloc[0, 0] 

    plt.figure()
    
    if ptype == 'line':
        touch_df = sensor_df.copy()
        non_touch_df = sensor_df.copy()
        
        touch_df[touch_df['touch'] == 0] = np.nan
        non_touch_df[touch_df['touch'] == 1] = np.nan
        
        plt.plot(touch_df['timestamp'], amp(touch_df), c='r')
        plt.plot(non_touch_df['timestamp'], amp(non_touch_df), c='b')
    else:
        plt.scatter(sensor_df['timestamp'], amp(sensor_df), c=sensor_df[2])

    plt.xlabel('Timestamp')
    plt.ylabel("{} (Amplitude)".format(name))
    
    plt.show()


def downsampling(df, rate):
    dfs = []
    
    for sensor in ZKSENSORS:
        sdf = df[df['sensor'] == sensor]
        
        sdf = sdf.set_index(pd.to_datetime(sdf.timestamp, unit='ms'))
        sdf = sdf.groupby(sdf.index).last()

        sdf = sdf.resample(rate).last().ffill()
        sdf['timestamp'] = sdf.index.values.astype(np.int64) // 10 ** 6
        dfs.append(sdf)
    
    ddf = pd.concat(dfs)
    
    if any(ddf['touch'] > 0):
        return ddf
    else:
        return ddf[0:0]


def get_click_segs(df, downsample_rate=None):
    sdf = df.loc[df['sensor'].isin(ZKSENSORS)]
    tdf = df.loc[df['sensor'] == 'TOU']
    sdf = sdf.reset_index(drop=True)
#     sdf['timestamp'] = sdf['timestamp'] - sdf.iloc[0, 0]

    groups = mit.consecutive_groups(sdf[sdf['touch'] == 1].index)

    click_seg_candidates = []
    click_segs = []
    
    for group in groups:
        g = list(group)
        
        click_seg = sdf.iloc[g[0]:(g[-1] + 1), :]
        
        start_time = click_seg.iloc[0, 0]
        end_time = click_seg.iloc[-1, 0]
        
        # The observed duration for a single short click is within the `TIME_DURATION`
        if end_time - start_time < TIME_DURATION:
            click_seg_candidates.append(click_seg)

    for i, s in enumerate(click_seg_candidates):
        if i == 0:
            pt = sdf.iloc[0, 0]
        else:
            pt = click_seg_candidates[i-1].iloc[-1, 0]
        
        if i < len(click_seg_candidates) - 1:
            nt = click_seg_candidates[i+1].iloc[0, 0]
        else:
            nt = sdf.iloc[-1, 0]

        if s.iloc[0, 0] - TIME_BEFORE > pt and s.iloc[-1, 0] + TIME_AFTER < nt:
            start_time = s.iloc[0, 0] - TIME_BEFORE if s.iloc[0, 0] - TIME_BEFORE > sdf.iloc[0, 0] else sdf.iloc[0, 0]
            end_time = s.iloc[-1, 0] + TIME_AFTER
            
            click_seg = sdf.loc[(sdf['timestamp'] >= start_time) & (sdf['timestamp'] <= end_time)]
            click_seg = click_seg.reset_index(drop=True)
            
            tdf_seg = tdf.loc[(tdf['timestamp'] >= start_time) & (tdf['timestamp'] <= end_time)]
            tdf_value_counts = tdf_seg['touch'].value_counts()
            
            if tdf_value_counts.get(1, 0) == 1 and tdf_value_counts.get(3, 0) == 1 and tdf_seg.iloc[-1,0] - tdf_seg.iloc[0,0] < TIME_DURATION:
            
#             if tdf_seg['d1'].nunique() == 1 and tdf_seg['d2'].nunique() == 1:
                tdf_seg = tdf_seg.reset_index(drop=True)

                click_seg['amp'] = amp(click_seg)
        
                if downsample_rate:
                    click_seg = downsampling(click_seg, downsample_rate)
                
                if not click_seg.empty:
                    click_segs.append((click_seg, tdf_seg))
        
    return click_segs


def get_drag_segs(df, downsample_rate=None):
    sdf = df.loc[df['sensor'].isin(ZKSENSORS)]
    tdf = df.loc[df['sensor'] == 'TOU']
    sdf = sdf.reset_index(drop=True)
#     sdf['timestamp'] = sdf['timestamp'] - sdf.iloc[0, 0]

    groups = mit.consecutive_groups(sdf[sdf['touch'] == 1].index)

    click_seg_candidates = []
    click_segs = []
    
    for group in groups:
        g = list(group)
        
        click_seg = sdf.iloc[g[0]:(g[-1] + 1), :]
        
        start_time = click_seg.iloc[0, 0]
        end_time = click_seg.iloc[-1, 0]
        
        # The observed duration for a single short click is within the `TIME_DURATION`
        if end_time - start_time < TIME_DURATION:
            click_seg_candidates.append(click_seg)

    for i, s in enumerate(click_seg_candidates):
        if i == 0:
            pt = sdf.iloc[0, 0]
        else:
            pt = click_seg_candidates[i-1].iloc[-1, 0]
        
        if i < len(click_seg_candidates) - 1:
            nt = click_seg_candidates[i+1].iloc[0, 0]
        else:
            nt = sdf.iloc[-1, 0]

        if s.iloc[0, 0] - TIME_BEFORE > pt and s.iloc[-1, 0] + TIME_AFTER < nt:
            start_time = s.iloc[0, 0] - TIME_BEFORE if s.iloc[0, 0] - TIME_BEFORE > sdf.iloc[0, 0] else sdf.iloc[0, 0]
            end_time = s.iloc[-1, 0] + TIME_AFTER
            
            click_seg = sdf.loc[(sdf['timestamp'] >= start_time) & (sdf['timestamp'] <= end_time)]
            click_seg = click_seg.reset_index(drop=True)
            
            tdf_seg = tdf.loc[(tdf['timestamp'] >= start_time) & (tdf['timestamp'] <= end_time)]
            tdf_value_counts = tdf_seg['touch'].value_counts()
            
            if tdf_value_counts.get(1, 0) == 1 and tdf_value_counts.get(3, 0) == 1 and tdf_seg.iloc[-1,0] - tdf_seg.iloc[0,0] < TIME_DURATION:
            
#             if tdf_seg['d1'].nunique() == 1 and tdf_seg['d2'].nunique() == 1:
                tdf_seg = tdf_seg.reset_index(drop=True)

                click_seg['amp'] = amp(click_seg)
        
                if downsample_rate:
                    click_seg = downsampling(click_seg, downsample_rate)
                
                if not click_seg.empty:
                    click_segs.append((click_seg, tdf_seg))
        
    return click_segs


def extract_features(click_segs, label, remark):
    features_dict = {
        "uac_min_x": [],
        "uac_min_y": [],
        "uac_min_z": [],
        "uac_min_amp": [],
        "uac_max_x": [],
        "uac_max_y": [],
        "uac_max_z": [],
        "uac_max_amp": [],
        "uac_gap_x": [],
        "uac_gap_y": [],
        "uac_gap_z": [],
        "uac_gap_amp": [],
        "uac_mean1_x": [],
        "uac_mean1_y": [],
        "uac_mean1_z": [],
        "uac_mean1_amp": [],
        "uac_mean2_x": [],
        "uac_mean2_y": [],
        "uac_mean2_z": [],
        "uac_mean2_amp": [],
        "uac_std1_x": [],
        "uac_std1_y": [],
        "uac_std1_z": [],
        "uac_std1_amp": [],
        "uac_std2_x": [],
        "uac_std2_y": [],
        "uac_std2_z": [],
        "uac_std2_amp": [],
        "uac_dev_min_x": [],
        "uac_dev_min_y": [],
        "uac_dev_min_z": [],
        "uac_dev_min_amp": [],
        "uac_dev_max_x": [],
        "uac_dev_max_y": [],
        "uac_dev_max_z": [],
        "uac_dev_max_amp": [],
        "uac_dev_gap_x": [],
        "uac_dev_gap_y": [],
        "uac_dev_gap_z": [],
        "uac_dev_gap_amp": [],
        "uac_dev_mean1_x": [],
        "uac_dev_mean1_y": [],
        "uac_dev_mean1_z": [],
        "uac_dev_mean1_amp": [],
        "uac_dev_mean2_x": [],
        "uac_dev_mean2_y": [],
        "uac_dev_mean2_z": [],
        "uac_dev_mean2_amp": [],
        "uac_dev_std1_x": [],
        "uac_dev_std1_y": [],
        "uac_dev_std1_z": [],
        "uac_dev_std1_amp": [],
        "uac_dev_std2_x": [],
        "uac_dev_std2_y": [],
        "uac_dev_std2_z": [],
        "uac_dev_std2_amp": [],
        "gyr_min_x": [],
        "gyr_min_y": [],
        "gyr_min_z": [],
        "gyr_min_amp": [],
        "gyr_max_x": [],
        "gyr_max_y": [],
        "gyr_max_z": [],
        "gyr_max_amp": [],
        "gyr_gap_x": [],
        "gyr_gap_y": [],
        "gyr_gap_z": [],
        "gyr_gap_amp": [],
        "gyr_mean1_x": [],
        "gyr_mean1_y": [],
        "gyr_mean1_z": [],
        "gyr_mean1_amp": [],
        "gyr_mean2_x": [],
        "gyr_mean2_y": [],
        "gyr_mean2_z": [],
        "gyr_mean2_amp": [],
        "gyr_std1_x": [],
        "gyr_std1_y": [],
        "gyr_std1_z": [],
        "gyr_std1_amp": [],
        "gyr_std2_x": [],
        "gyr_std2_y": [],
        "gyr_std2_z": [],
        "gyr_std2_amp": [],
        "gyr_dev_min_x": [],
        "gyr_dev_min_y": [],
        "gyr_dev_min_z": [],
        "gyr_dev_min_amp": [],
        "gyr_dev_max_x": [],
        "gyr_dev_max_y": [],
        "gyr_dev_max_z": [],
        "gyr_dev_max_amp": [],
        "gyr_dev_gap_x": [],
        "gyr_dev_gap_y": [],
        "gyr_dev_gap_z": [],
        "gyr_dev_gap_amp": [],
        "gyr_dev_mean1_x": [],
        "gyr_dev_mean1_y": [],
        "gyr_dev_mean1_z": [],
        "gyr_dev_mean1_amp": [],
        "gyr_dev_mean2_x": [],
        "gyr_dev_mean2_y": [],
        "gyr_dev_mean2_z": [],
        "gyr_dev_mean2_amp": [],
        "gyr_dev_std1_x": [],
        "gyr_dev_std1_y": [],
        "gyr_dev_std1_z": [],
        "gyr_dev_std1_amp": [],
        "gyr_dev_std2_x": [],
        "gyr_dev_std2_y": [],
        "gyr_dev_std2_z": [],
        "gyr_dev_std2_amp": [],
        "touch_x" : [],
        "touch_y" : [],
        "touch_duration": [],
        "click_duration": [],
        "uac_num1": [],
        "uac_num2": [],
        "gyr_num1": [],
        "gyr_num2": [],
        "label": [],
        "remark": []
    }
    
    for cs, ts in click_segs:
        cs1 = cs[cs['timestamp'] <= ts.iloc[-1, 0] + TIME_BEFORE]
        cs2 = cs[cs['timestamp'] > ts.iloc[-1, 0]]
        
        if cs1[cs1['sensor'] == 'UAC'].shape[0] < MINIMAL_POINTS or cs1[cs1['sensor'] == 'GYR'].shape[0] < MINIMAL_POINTS or cs2[cs2['sensor'] == 'UAC'].shape[0] < MINIMAL_POINTS or cs2[cs2['sensor'] == 'GYR'].shape[0] < MINIMAL_POINTS:
            continue
        
        for sensor in ZKSENSORS:
            css1 = cs1[cs1['sensor'] == sensor]
            css1 = css1.reset_index(drop=True)
            css2 = cs2[cs2['sensor'] == sensor]
            css2 = css2.reset_index(drop=True)
            
            css1_dtime = css1[1:]['timestamp'].values - css1[:-1]['timestamp'].values
            css1_dtime = np.where(css1_dtime==0, 0.5, css1_dtime)
            
            css2_dtime = css2[1:]['timestamp'].values - css2[:-1]['timestamp'].values
            css2_dtime = np.where(css2_dtime==0, 0.5, css2_dtime)
                
            for axis in ['x', 'y', 'z', 'amp']:
                axis_index = AXIS_INDEX[axis]
                
                min_idx = css1[axis_index].idxmin()
                max_idx = css1[axis_index].idxmax()
                min_v = css1.loc[min_idx, axis_index]
                max_v = css1.loc[max_idx, axis_index]

                features_dict[sensor.lower() + "_min_" + axis].append(min_v)
                features_dict[sensor.lower() + "_max_" + axis].append(max_v)
#                 features_dict[sensor.lower() + "_diff_" + axis].append(max_v - min_v)
                features_dict[sensor.lower() + "_gap_" + axis].append(abs(css1.iloc[max_idx, 0] - css1.iloc[min_idx, 0]))
                features_dict[sensor.lower() + "_mean1_" + axis].append(css1[axis_index].mean())
                features_dict[sensor.lower() + "_mean2_" + axis].append(css2[axis_index].mean())
#                 features_dict[sensor.lower() + "_cv1_" + axis].append(variation(css1[axis_index]))
#                 features_dict[sensor.lower() + "_cv2_" + axis].append(variation(css2[axis_index]))
                features_dict[sensor.lower() + "_std1_" + axis].append(css1[axis_index].std())
                features_dict[sensor.lower() + "_std2_" + axis].append(css2[axis_index].std())
        
#                 css1_dev = (css1[1:][axis_index].values - css1[:-1][axis_index].values) / css1_dtime
#                 css2_dev = (css2[1:][axis_index].values - css2[:-1][axis_index].values) / css2_dtime
                
                css1_dev = (css1[1:][axis_index].values - css1[:-1][axis_index].values)
                css2_dev = (css2[1:][axis_index].values - css2[:-1][axis_index].values)
                
                min_idx = np.argmin(css1_dev)
                max_idx = np.argmax(css1_dev)
                min_v = css1_dev[min_idx]
                max_v = css1_dev[max_idx]
                
                features_dict[sensor.lower() + "_dev_min_" + axis].append(min_v)
                features_dict[sensor.lower() + "_dev_max_" + axis].append(max_v)
                features_dict[sensor.lower() + "_dev_gap_" + axis].append(abs(css1.iloc[max_idx + 1, 0] - css1.iloc[min_idx + 1, 0]))
                features_dict[sensor.lower() + "_dev_mean1_" + axis].append(css1_dev.mean())
                features_dict[sensor.lower() + "_dev_mean2_" + axis].append(css2_dev.mean())
                features_dict[sensor.lower() + "_dev_std1_" + axis].append(css1_dev.std())
                features_dict[sensor.lower() + "_dev_std2_" + axis].append(css2_dev.std())
            
            features_dict[sensor.lower() + "_num1"].append(css1.shape[0])
            features_dict[sensor.lower() + "_num2"].append(css2.shape[0])
            
            
        features_dict["touch_x"].append(int(ts['d1'].mean()))
        features_dict["touch_y"].append(int(ts['d2'].mean()))
        features_dict["touch_duration"].append(ts.iloc[-1, 0] - ts.iloc[0, 0])
        features_dict["click_duration"].append(cs.iloc[-1, 0] - cs.iloc[0, 0])
        features_dict["label"].append(label)
        features_dict["remark"].append(remark)
    
    return pd.DataFrame(data=features_dict)


def not_enough_points(cs, minimal_points=MINIMAL_POINTS):
    return cs[cs['sensor'] == 'UAC'].shape[0] < minimal_points or cs[cs['sensor'] == 'GYR'].shape[0] < minimal_points


def extract_features_three_segs(click_segs, label, remark):
    features_dict = {
        "uac_min_x": [],
        "uac_min_y": [],
        "uac_min_z": [],
        "uac_min_amp": [],
        "uac_max_x": [],
        "uac_max_y": [],
        "uac_max_z": [],
        "uac_max_amp": [],
        "uac_gap_x": [],
        "uac_gap_y": [],
        "uac_gap_z": [],
        "uac_gap_amp": [],
        "uac_mean1_x": [],
        "uac_mean1_y": [],
        "uac_mean1_z": [],
        "uac_mean1_amp": [],
        "uac_mean2_x": [],
        "uac_mean2_y": [],
        "uac_mean2_z": [],
        "uac_mean2_amp": [],
        "uac_mean3_x": [],
        "uac_mean3_y": [],
        "uac_mean3_z": [],
        "uac_mean3_amp": [],
        "uac_std1_x": [],
        "uac_std1_y": [],
        "uac_std1_z": [],
        "uac_std1_amp": [],
        "uac_std2_x": [],
        "uac_std2_y": [],
        "uac_std2_z": [],
        "uac_std2_amp": [],
        "uac_std3_x": [],
        "uac_std3_y": [],
        "uac_std3_z": [],
        "uac_std3_amp": [],
        "uac_dev_min_x": [],
        "uac_dev_min_y": [],
        "uac_dev_min_z": [],
        "uac_dev_min_amp": [],
        "uac_dev_max_x": [],
        "uac_dev_max_y": [],
        "uac_dev_max_z": [],
        "uac_dev_max_amp": [],
        "uac_dev_gap_x": [],
        "uac_dev_gap_y": [],
        "uac_dev_gap_z": [],
        "uac_dev_gap_amp": [],
        "uac_dev_mean1_x": [],
        "uac_dev_mean1_y": [],
        "uac_dev_mean1_z": [],
        "uac_dev_mean1_amp": [],
        "uac_dev_mean2_x": [],
        "uac_dev_mean2_y": [],
        "uac_dev_mean2_z": [],
        "uac_dev_mean2_amp": [],
        "uac_dev_mean3_x": [],
        "uac_dev_mean3_y": [],
        "uac_dev_mean3_z": [],
        "uac_dev_mean3_amp": [],
        "uac_dev_std1_x": [],
        "uac_dev_std1_y": [],
        "uac_dev_std1_z": [],
        "uac_dev_std1_amp": [],
        "uac_dev_std2_x": [],
        "uac_dev_std2_y": [],
        "uac_dev_std2_z": [],
        "uac_dev_std2_amp": [],
        "uac_dev_std3_x": [],
        "uac_dev_std3_y": [],
        "uac_dev_std3_z": [],
        "uac_dev_std3_amp": [],
        "gyr_min_x": [],
        "gyr_min_y": [],
        "gyr_min_z": [],
        "gyr_min_amp": [],
        "gyr_max_x": [],
        "gyr_max_y": [],
        "gyr_max_z": [],
        "gyr_max_amp": [],
        "gyr_gap_x": [],
        "gyr_gap_y": [],
        "gyr_gap_z": [],
        "gyr_gap_amp": [],
        "gyr_mean1_x": [],
        "gyr_mean1_y": [],
        "gyr_mean1_z": [],
        "gyr_mean1_amp": [],
        "gyr_mean2_x": [],
        "gyr_mean2_y": [],
        "gyr_mean2_z": [],
        "gyr_mean2_amp": [],
        "gyr_mean3_x": [],
        "gyr_mean3_y": [],
        "gyr_mean3_z": [],
        "gyr_mean3_amp": [],
        "gyr_std1_x": [],
        "gyr_std1_y": [],
        "gyr_std1_z": [],
        "gyr_std1_amp": [],
        "gyr_std2_x": [],
        "gyr_std2_y": [],
        "gyr_std2_z": [],
        "gyr_std2_amp": [],
        "gyr_std3_x": [],
        "gyr_std3_y": [],
        "gyr_std3_z": [],
        "gyr_std3_amp": [],
        "gyr_dev_min_x": [],
        "gyr_dev_min_y": [],
        "gyr_dev_min_z": [],
        "gyr_dev_min_amp": [],
        "gyr_dev_max_x": [],
        "gyr_dev_max_y": [],
        "gyr_dev_max_z": [],
        "gyr_dev_max_amp": [],
        "gyr_dev_gap_x": [],
        "gyr_dev_gap_y": [],
        "gyr_dev_gap_z": [],
        "gyr_dev_gap_amp": [],
        "gyr_dev_mean1_x": [],
        "gyr_dev_mean1_y": [],
        "gyr_dev_mean1_z": [],
        "gyr_dev_mean1_amp": [],
        "gyr_dev_mean2_x": [],
        "gyr_dev_mean2_y": [],
        "gyr_dev_mean2_z": [],
        "gyr_dev_mean2_amp": [],
        "gyr_dev_mean3_x": [],
        "gyr_dev_mean3_y": [],
        "gyr_dev_mean3_z": [],
        "gyr_dev_mean3_amp": [],
        "gyr_dev_std1_x": [],
        "gyr_dev_std1_y": [],
        "gyr_dev_std1_z": [],
        "gyr_dev_std1_amp": [],
        "gyr_dev_std2_x": [],
        "gyr_dev_std2_y": [],
        "gyr_dev_std2_z": [],
        "gyr_dev_std2_amp": [],
        "gyr_dev_std3_x": [],
        "gyr_dev_std3_y": [],
        "gyr_dev_std3_z": [],
        "gyr_dev_std3_amp": [],
        "touch_x" : [],
        "touch_y" : [],
        "touch_duration": [],
        "uac_num1": [],
        "uac_num2": [],
        "uac_num3": [],
        "gyr_num1": [],
        "gyr_num2": [],
        "gyr_num3": [],
        "label": [],
        "remark": []
    }
    
    for cs, ts in click_segs:
        cs1 = cs[cs['timestamp'] <= ts.iloc[0, 0] + 20]
        cs2 = cs[(cs['timestamp'] > ts.iloc[0, 0] + 20) & (cs['timestamp'] <= ts.iloc[-1, 0])]
        cs3 = cs[cs['timestamp'] > ts.iloc[-1, 0]]
        
        if not_enough_points(cs1) or not_enough_points(cs2) or not_enough_points(cs3):
            continue
        
        for sensor in ZKSENSORS:
            css1 = cs1[cs1['sensor'] == sensor]
            css1 = css1.reset_index(drop=True)
            css2 = cs2[cs2['sensor'] == sensor]
            css2 = css2.reset_index(drop=True)
            css3 = cs3[cs3['sensor'] == sensor]
            css3 = css3.reset_index(drop=True)
            
            css1_dtime = css1[1:]['timestamp'].values - css1[:-1]['timestamp'].values
            css1_dtime = np.where(css1_dtime==0, 0.5, css1_dtime)
            
            css2_dtime = css2[1:]['timestamp'].values - css2[:-1]['timestamp'].values
            css2_dtime = np.where(css2_dtime==0, 0.5, css2_dtime)
            
            css3_dtime = css3[1:]['timestamp'].values - css3[:-1]['timestamp'].values
            css3_dtime = np.where(css3_dtime==0, 0.5, css3_dtime)
                
            for axis in ['x', 'y', 'z', 'amp']:
                axis_index = AXIS_INDEX[axis]
                
                min_idx = css1[axis_index].idxmin()
                max_idx = css1[axis_index].idxmax()
                min_v = css1.loc[min_idx, axis_index]
                max_v = css1.loc[max_idx, axis_index]

                features_dict[sensor.lower() + "_min_" + axis].append(min_v)
                features_dict[sensor.lower() + "_max_" + axis].append(max_v)
#                 features_dict[sensor.lower() + "_diff_" + axis].append(max_v - min_v)
                features_dict[sensor.lower() + "_gap_" + axis].append(abs(css1.iloc[max_idx, 0] - css1.iloc[min_idx, 0]))
                features_dict[sensor.lower() + "_mean1_" + axis].append(css1[axis_index].mean())
                features_dict[sensor.lower() + "_mean2_" + axis].append(css2[axis_index].mean())
                features_dict[sensor.lower() + "_mean3_" + axis].append(css3[axis_index].mean())
#                 features_dict[sensor.lower() + "_cv1_" + axis].append(variation(css1[axis_index]))
#                 features_dict[sensor.lower() + "_cv2_" + axis].append(variation(css2[axis_index]))
                features_dict[sensor.lower() + "_std1_" + axis].append(css1[axis_index].std())
                features_dict[sensor.lower() + "_std2_" + axis].append(css2[axis_index].std())
                features_dict[sensor.lower() + "_std3_" + axis].append(css3[axis_index].std())
        
#                 css1_dev = (css1[1:][axis_index].values - css1[:-1][axis_index].values) / css1_dtime
#                 css2_dev = (css2[1:][axis_index].values - css2[:-1][axis_index].values) / css2_dtime
#                 css3_dev = (css3[1:][axis_index].values - css3[:-1][axis_index].values) / css3_dtime

                css1_dev = (css1[1:][axis_index].values - css1[:-1][axis_index].values)
                css2_dev = (css2[1:][axis_index].values - css2[:-1][axis_index].values)
                css3_dev = (css3[1:][axis_index].values - css3[:-1][axis_index].values)
                
                min_idx = np.argmin(css1_dev)
                max_idx = np.argmax(css1_dev)
                min_v = css1_dev[min_idx]
                max_v = css1_dev[max_idx]
                
                features_dict[sensor.lower() + "_dev_min_" + axis].append(min_v)
                features_dict[sensor.lower() + "_dev_max_" + axis].append(max_v)
                features_dict[sensor.lower() + "_dev_gap_" + axis].append(abs(css1.iloc[max_idx + 1, 0] - css1.iloc[min_idx + 1, 0]))
                features_dict[sensor.lower() + "_dev_mean1_" + axis].append(css1_dev.mean())
                features_dict[sensor.lower() + "_dev_mean2_" + axis].append(css2_dev.mean())
                features_dict[sensor.lower() + "_dev_mean3_" + axis].append(css3_dev.mean())
                features_dict[sensor.lower() + "_dev_std1_" + axis].append(css1_dev.std())
                features_dict[sensor.lower() + "_dev_std2_" + axis].append(css2_dev.std())
                features_dict[sensor.lower() + "_dev_std3_" + axis].append(css3_dev.std())
            
            features_dict[sensor.lower() + "_num1"].append(css1.shape[0])
            features_dict[sensor.lower() + "_num2"].append(css2.shape[0])
            features_dict[sensor.lower() + "_num3"].append(css3.shape[0])
            
            
        features_dict["touch_x"].append(int(ts['d1'].mean()))
        features_dict["touch_y"].append(int(ts['d2'].mean()))
        features_dict["touch_duration"].append(ts.iloc[-1, 0] - ts.iloc[0, 0])
#         features_dict["click_duration"].append(cs.iloc[-1, 0] - cs.iloc[0, 0])
        features_dict["label"].append(label)
        features_dict["remark"].append(remark)
    
    return pd.DataFrame(data=features_dict)


def extract_features_for_folder(folder_path, label, remark, segs=2, to_csv=True, output_folder="data/LabelledData", output_file=None, replace=True):
    features = []
    
    if not output_file:
        output_file = "{}/{}-{}.csv".format(output_folder, label, remark)
    
    if not replace:
        if os.path.exists(output_file):
            return
    
    for f in glob2.glob("{}/**/*.csv".format(folder_path)):
        df = read_csv(f)
        click_segs = get_click_segs(df, DOWNSAMPLE_RATE)
        
        feature = extract_features(click_segs, label, remark) if segs == 2 else extract_features_three_segs(click_segs, label, remark)        
        features.append(feature)

    feature_df = pd.concat(features, ignore_index=True)
    feature_df.dropna(inplace=True)
    feature_df.reset_index(drop=True, inplace=True)
    
    if to_csv:
        feature_df.to_csv(output_file)
        print("\nLabelled data is stored at: {}\n".format(output_file))
        print(feature_df.head())
    else:
        print(feature_df)
        
        

In [49]:
plot_for_slides_diff()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Extract features from all files

## Lion (Human)

In [92]:
# Export labelled data for all users

human_data_folder = "data/Android/Processed"

for d in glob2.glob('{}/*/'.format(human_data_folder)):
    remark = d.split("/")[-2]
    extract_features_for_folder(d[:-1], "Human", remark, output_folder="data/LabelledDataDiff", replace=True)


Labelled data is stored at: data/LabelledDataDiff/Human-Tony.csv

      uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -3.120700e+09  5.462000e+08 -1.023820e+10  8.676319e+09 -2.620400e+09   
1 -2.780100e+09  3.004000e+08 -1.084920e+10  8.148870e+09 -1.751800e+09   
2 -3.462400e+09  2.240000e+09 -8.436800e+09  8.359443e+09 -2.816500e+09   
3 -3.357000e+09  2.237200e+09 -9.122399e+09  7.975604e+09 -2.812600e+09   
4 -3.705100e+09  9.972000e+08 -1.116940e+10  8.422227e+09 -2.919600e+09   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  9.750000e+08 -8.226600e+09  1.065478e+10       36.0       12.0  ...   
1  1.443100e+09 -7.950800e+09  1.123667e+10       48.0       24.0  ...   
2  2.751900e+09 -7.482100e+09  9.369225e+09       68.0       20.0  ...   
3  2.759200e+09 -7.100000e+09  9.988985e+09      144.0       80.0  ...   
4  1.622200e+09 -7.836800e+09  1.176902e+10       48.0       32.0  ...   

        touch_x       touch_y  touch_


Labelled data is stored at: data/LabelledDataDiff/Human-Inigo.csv

     uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -253600270.0 -1.364656e+08 -1.397464e+09  7.775297e+07  5.211947e+08   
1 -609038600.0 -2.360568e+09 -4.469114e+09  5.482176e+08  1.359809e+09   
2 -148074870.0 -1.683559e+08 -9.061122e+08  4.389745e+07  2.842148e+08   
3 -319619660.0 -1.741428e+08 -1.245811e+09  1.252904e+08  4.741986e+08   
4 -154422580.0 -1.364718e+08 -2.148447e+08  2.231423e+07  8.256314e+07   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  2.781925e+08  6.485529e+08  1.464956e+09       72.0       80.0  ...   
1  1.773975e+09  2.285557e+09  4.888783e+09       36.0      108.0  ...   
2  2.726102e+08  4.618006e+08  9.157001e+08       20.0       16.0  ...   
3  2.112756e+08  3.675938e+08  1.337839e+09      100.0       12.0  ...   
4  1.127701e+08  3.630552e+08  4.103322e+08        4.0        4.0  ...   

        touch_x       touch_y  touch_durat

In [96]:
# Export labelled data for three segments

human_data_folder = "data/Android/Processed"

for d in glob2.glob('{}/*/'.format(human_data_folder)):
    remark = d.split("/")[-2]
    extract_features_for_folder(d[:-1], "Human", remark, segs=3, output_folder="data/LabelledData3SegsDiff", replace=True)


Labelled data is stored at: data/LabelledData3SegsDiff/Human-Tony.csv

      uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -3.120700e+09  5.462000e+08 -1.023820e+10  8.676319e+09 -2.689400e+09   
1 -2.780100e+09  3.004000e+08 -1.084920e+10  8.266414e+09 -1.807000e+09   
2 -3.255300e+09  2.339400e+09 -8.280000e+09  8.359443e+09 -2.816500e+09   
3 -3.357000e+09  2.253200e+09 -8.873700e+09  8.639865e+09 -2.858700e+09   
4 -3.705100e+09  9.972000e+08 -1.116940e+10  8.422227e+09 -2.919600e+09   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  9.750000e+08 -8.226600e+09  1.065478e+10       32.0       12.0  ...   
1  1.443100e+09 -8.060900e+09  1.123667e+10       36.0       24.0  ...   
2  2.574000e+09 -7.482100e+09  9.198405e+09       28.0       24.0  ...   
3  2.759200e+09 -7.835700e+09  9.777880e+09       44.0       48.0  ...   
4  1.622200e+09 -7.836800e+09  1.176902e+10       48.0       32.0  ...   

        touch_y  touch_duration 


Labelled data is stored at: data/LabelledData3SegsDiff/Human-Inigo.csv

     uac_min_x    uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0  -62513830.0  -67409515.0 -1.397464e+09  7.775297e+07  5.211947e+08   
1   54943323.0  150044440.0 -4.469114e+09  5.482176e+08  1.295134e+09   
2  -19202470.0  -84265230.0 -9.061122e+08  4.801259e+07  2.842148e+08   
3  150639410.0 -174142840.0 -1.245811e+09  1.891075e+08  4.741986e+08   
4 -154422580.0 -136471750.0 -2.148447e+08  1.629050e+08  8.256314e+07   

      uac_max_y    uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  2.781925e+08   26679993.0  1.464956e+09       12.0       12.0  ...   
1  1.773975e+09 -145078660.0  4.888783e+09       48.0       56.0  ...   
2  8.473921e+07  169639590.0  9.157001e+08       32.0        8.0  ...   
3  2.112756e+08  183400630.0  1.337839e+09       36.0       12.0  ...   
4  1.127701e+08  363055230.0  4.103322e+08        4.0        4.0  ...   

        touch_y  touch_duration  uac_num1  uac_nu

In [5]:
# Export labelled data for a single user

data_folder = "data/Android/Processed/Tony"
extract_features_for_folder(data_folder, "Human", "Tony", replace=True)


Labelled data is stored at: data/LabelledData/Human-Tony.csv

      uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -3.120700e+09  5.462000e+08 -1.023820e+10  8.676319e+09 -2.620400e+09   
1 -2.780100e+09  3.004000e+08 -1.084920e+10  8.148870e+09 -1.751800e+09   
2 -3.462400e+09  2.240000e+09 -8.436800e+09  8.359443e+09 -2.816500e+09   
3 -3.357000e+09  2.237200e+09 -9.122399e+09  7.975604e+09 -2.812600e+09   
4 -3.705100e+09  9.972000e+08 -1.116940e+10  8.422227e+09 -2.919600e+09   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  9.750000e+08 -8.226600e+09  1.065478e+10       36.0       12.0  ...   
1  1.443100e+09 -7.950800e+09  1.123667e+10       48.0       24.0  ...   
2  2.751900e+09 -7.482100e+09  9.369225e+09       68.0       20.0  ...   
3  2.759200e+09 -7.100000e+09  9.988985e+09      144.0       80.0  ...   
4  1.622200e+09 -7.836800e+09  1.176902e+10       48.0       32.0  ...   

        touch_x       touch_y  touch_dura

## Lion (Bot)

In [93]:
# Export labelled data for all users

human_data_folder = "data/Bot/data"

for d in glob2.glob('{}/*/'.format(human_data_folder)):
    remark = d.split("/")[-2]
    extract_features_for_folder(d[:-1], "Bot", remark, output_folder="data/LabelledDataDiff", replace=True)


Labelled data is stored at: data/LabelledDataDiff/Bot-Tony.csv

    uac_min_x   uac_min_y    uac_min_z   uac_min_amp   uac_max_x   uac_max_y  \
0 -11500000.0 -16800000.0  -48200000.0  3.090938e+07  11700000.0  32199997.0   
1 -26999999.0 -49700000.0  -18600000.0  2.747253e+07  -4600000.0 -18099999.0   
2 -19299999.0 -29099999.0  -38200000.0  8.704596e+06  10300000.0    600000.0   
3 -17400000.0 -32600000.0 -106699996.0  5.436414e+07  21300000.0  -2600000.0   
4 -36300000.0 -43399997.0 -145900000.0  7.095830e+07   2700000.0 -12399999.0   

    uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...       touch_x  \
0 -18700000.0  4.976434e+07         32        192  ...  978000000000   
1  49999997.0  6.843398e+07         68         28  ...  881000000000   
2    900000.0  4.061391e+07        108         96  ...  103000000000   
3 -47900000.0  1.074744e+08         80         60  ...  196000000000   
4 -67400000.0  1.490988e+08         12         48  ...  981000000000   

         touch_y  tou


Labelled data is stored at: data/LabelledDataDiff/Bot-Tony-Handheld.csv

      uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0  8.239000e+08  3.159000e+08 -1.419000e+09  1.314717e+09  2.303600e+09   
1 -5.053000e+08  4.513000e+08 -2.090000e+09  1.417109e+09  1.486000e+09   
2 -6.440000e+07  7.603000e+08 -3.108300e+09  1.127015e+09  9.035000e+08   
3 -1.192700e+09  6.600000e+08 -2.406600e+09  1.031824e+09  1.504600e+09   
4 -2.000000e+09 -1.746400e+09 -3.490600e+09  1.541359e+09  3.285000e+08   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  1.381500e+09  4.943000e+08  2.491842e+09         24         76  ...   
1  1.934500e+09 -6.004000e+08  2.892355e+09        148        156  ...   
2  1.603800e+09 -6.045000e+08  3.480265e+09        132         48  ...   
3  1.879200e+09  1.362100e+09  2.913646e+09        116        188  ...   
4  2.686400e+09  1.776900e+09  4.837461e+09         44         88  ...   

         touch_x        touch_


Labelled data is stored at: data/LabelledDataDiff/Bot-Stan-S8-Swing.csv

     uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -409381840.0 -4.666049e+09 -5.466220e+09  4.462146e+09  1.163372e+09   
1 -218611630.0 -4.694942e+09 -4.877338e+09  3.775003e+09  7.826184e+08   
2   59008062.0 -2.630224e+09  3.591066e+09  4.076582e+09  5.384038e+08   
3 -270026330.0 -4.507768e+09 -4.064135e+09  3.836504e+09  6.731878e+08   
4 -134505030.0 -4.553377e+09 -5.056323e+09  3.697940e+09  9.472228e+08   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0 -2.936548e+09 -1.739241e+09  6.816541e+09         56        160  ...   
1 -3.426153e+09  3.981812e+09  6.200917e+09        164         60  ...   
2 -1.872702e+09  5.091562e+09  5.750863e+09         48         88  ...   
3 -2.392805e+09  3.626929e+09  5.460158e+09         64        116  ...   
4 -3.153956e+09  3.803005e+09  6.060245e+09        176        132  ...   

        touch_x        touch_y  touc


Labelled data is stored at: data/LabelledDataDiff/Bot-Stan-Train-Desk.csv

     uac_min_x    uac_min_y    uac_min_z   uac_min_amp    uac_max_x  \
0 -123866350.0 -319722530.0 -306420330.0  3.899707e+07  187541200.0   
1 -141407860.0 -453743520.0 -185535430.0  8.223774e+07   36461987.0   
2 -126089830.0 -615660370.0 -431081770.0  1.066897e+08  250231680.0   
3 -151193770.0 -718430640.0 -380228040.0  2.182762e+08   82102190.0   
4 -289310070.0 -327774050.0 -173598290.0  7.360558e+07   76251250.0   

     uac_max_y    uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  190370740.0  299331670.0  3.711082e+08         40         12  ...   
1   60778140.0  510402700.0  5.115899e+08        136        148  ...   
2  323007820.0  289018630.0  7.336197e+08        116         12  ...   
3  431348380.0  435122500.0  7.698602e+08         96         12  ...   
4  258856700.0  255407330.0  3.650954e+08         76        152  ...   

        touch_x        touch_y  touch_duration  click_duration  

In [98]:
# Export labelled data for all users

human_data_folder = "data/Bot/data"

for d in glob2.glob('{}/*/'.format(human_data_folder)):
    remark = d.split("/")[-2]
    extract_features_for_folder(d[:-1], "Bot", remark, segs=3, output_folder="data/LabelledData3SegsDiff", replace=True)


Labelled data is stored at: data/LabelledData3SegsDiff/Bot-Tony.csv

    uac_min_x   uac_min_y    uac_min_z   uac_min_amp   uac_max_x   uac_max_y  \
0 -11500000.0 -16800000.0  -38300000.0  3.090938e+07  11700000.0  12300000.0   
1 -20499999.0 -49700000.0   -8800000.0  2.747253e+07  -9699999.0 -18099999.0   
2 -10100000.0 -29099999.0  -18600000.0  9.188579e+06  10300000.0  -7999999.0   
3  -7999999.0 -23899999.0  -87100000.0  5.436414e+07  21300000.0 -14099999.0   
4 -36300000.0 -31900000.0 -145900000.0  7.095830e+07   2700000.0 -12399999.0   

    uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...        touch_y  \
0 -28500000.0  4.337488e+07         32         24  ...   224000000000   
1  40200000.0  6.470031e+07         28         28  ...  1334000000000   
2    900000.0  3.598305e+07         32         12  ...  1688000000000   
3 -47900000.0  8.896730e+07         44          8  ...   971000000000   
4 -67400000.0  1.490988e+08         12         64  ...   114000000000   

   touch_d


Labelled data is stored at: data/LabelledData3SegsDiff/Bot-Tony-Handheld.csv

      uac_min_x    uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -5.010000e+07  451300000.0 -1.579300e+09  1.417109e+09  1.486000e+09   
1  1.980000e+08  792899970.0 -2.247000e+09  1.342986e+09  9.035000e+08   
2 -4.497000e+08  659999970.0 -2.406600e+09  1.031824e+09  1.504600e+09   
3 -1.876500e+09 -852799950.0 -2.908400e+09  1.596478e+09 -6.379000e+08   
4 -9.498000e+08  756700000.0 -2.246200e+09  1.108290e+09 -3.693000e+08   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  1.516200e+09 -6.004000e+08  2.009535e+09         44         36  ...   
1  1.271300e+09 -1.065700e+09  2.638957e+09         60         40  ...   
2  1.123500e+09  1.362100e+09  2.740291e+09         44         60  ...   
3  2.148500e+09  1.290000e+08  3.058428e+09         60         40  ...   
4  9.366000e+08 -5.764000e+08  2.612423e+09         44         36  ...   

         touch_y  touch_duratio


Labelled data is stored at: data/LabelledData3SegsDiff/Bot-Stan-S8-Swing.csv

     uac_min_x     uac_min_y     uac_min_z   uac_min_amp     uac_max_x  \
0 -303211120.0 -4.666049e+09 -5.179585e+09  4.462146e+09  1.163372e+09   
1 -218611630.0 -3.980922e+09 -4.877338e+09  4.518949e+09  6.671909e+08   
2  263867860.0 -2.630224e+09  4.554608e+09  5.102067e+09  5.350788e+08   
3  -42096853.0 -4.507768e+09 -6.613596e+08  3.836504e+09  6.222560e+08   
4 -134505030.0 -3.778838e+09 -5.056323e+09  5.348504e+09  3.789507e+08   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0 -3.717916e+09 -1.739241e+09  6.714588e+09         32          4  ...   
1 -3.501430e+09 -2.559480e+09  6.200917e+09         60         16  ...   
2 -2.281900e+09  5.091562e+09  5.750863e+09         24         28  ...   
3 -3.834506e+09  3.626929e+09  5.460158e+09         40         12  ...   
4 -3.153956e+09 -3.863622e+09  6.060245e+09         32         40  ...   

         touch_y  touch_duratio


Labelled data is stored at: data/LabelledData3SegsDiff/Bot-Stan-Train-Desk.csv

     uac_min_x    uac_min_y    uac_min_z   uac_min_amp    uac_max_x  \
0 -123866350.0 -319722530.0 -210816380.0  1.196119e+08  187541200.0   
1 -115324540.0 -324284140.0   47690390.0  1.115857e+08  -49407900.0   
2  -73772740.0 -602123600.0 -431081770.0  1.066897e+08  198170970.0   
3  -71946440.0 -327774050.0 -140388490.0  7.360558e+07   60453802.0   
4 -360413250.0 -539553100.0 -467477800.0  2.102271e+08  249052610.0   

     uac_max_y    uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  109036030.0  299331670.0  3.711082e+08         40         12  ...   
1   60778140.0  214524270.0  3.363289e+08         12         12  ...   
2  129616560.0  264881130.0  6.539541e+08         12         12  ...   
3   77572525.0  102787970.0  3.347570e+08         28         40  ...   
4  362445270.0  285951600.0  6.509310e+08         36         12  ...   

         touch_y  touch_duration  uac_num1  uac_num2  uac_n

In [11]:
# Export labelled data for a single user

data_folder = "data/Bot/data/Tony-Swing"
extract_features_for_folder(data_folder, "Bot", "Tony-Swing")


Labelled data is stored at: data/LabelledData/Bot-Tony-Swing.csv

     uac_min_x     uac_min_y     uac_min_z   uac_min_amp    uac_max_x  \
0 -898700000.0 -6.633300e+09 -4.216000e+09  3.692247e+09  212900000.0   
1 -471100000.0 -8.998000e+08 -4.126800e+09  3.756422e+09 -142099990.0   
2 -754399960.0 -6.669600e+09 -3.708600e+09  3.527732e+09  405200000.0   
3 -689500000.0 -6.789500e+09 -5.867000e+08  4.705563e+09  239900000.0   
4 -273900000.0 -3.864200e+09  3.854300e+09  3.990465e+09  263600000.0   

      uac_max_y     uac_max_z   uac_max_amp  uac_gap_x  uac_gap_y  ...  \
0  7.531000e+08 -4.250000e+08  6.648491e+09         40        200  ...   
1  5.432000e+08 -3.634000e+09  4.165462e+09         80         68  ...   
2 -2.525000e+08  3.565500e+09  7.080438e+09         32        160  ...   
3 -1.968700e+09  4.347600e+09  6.967457e+09        164        148  ...   
4 -1.771000e+08  4.478400e+09  5.645622e+09         32        132  ...   

        touch_x        touch_y  touch_duration  c

# Data Analysis

In [15]:
df = read_csv("/Users/stanzhang/Developer/Brave/Analysis/Jupyter/data/Android/Processed/Tony/files/test-SNE_LX1-1564095058538.csv")

plot_sensor_axis("GYR", "y", df)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Click Duration Analysis 

In [122]:
human_data_folder = "data/Android/Processed"

durations = []

for f in glob2.glob(f'{human_data_folder}/**/*.csv'):
    df = read_csv(data_file)
    click_segs = get_click_segs(df)
    
    for s in click_segs:
        durations.append(s[1].loc[1,'timestamp'] - s[1].loc[0,'timestamp'])

# print(durations)

KeyboardInterrupt: 

In [ ]:
plt.hist(durations)

## Bot - Pixel 3 (Stan)

In [6]:
data_file = "data/Bot/data/Stan-Vibrate/android_build-Pixel_3-1565286415843.csv"

In [10]:
df = read_csv(data_file)

In [11]:
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3
0,1565286415848,GYR,0,0,0,-552296.0,1166851.0,-705296.0
1,1565286415850,GYR,0,0,0,1578236.0,2232102.0,-1770554.0
2,1565286415850,UAC,0,0,0,-1355916.0,24921805.0,-33300400.0
3,1565286415850,GYR,0,0,0,512978.0,1166826.0,-705284.0
4,1565286415851,GYR,0,0,0,-1617543.0,-963714.0,-705278.0


In [13]:
plot_sensor_axis("UAC", "z", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
click_segs = get_click_segs(df, 'UAC')

In [12]:
plot_sensor_axis("UAC", "z", df=click_segs[1][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Pixel 3 Position Compare

### Desk

In [23]:
data_file = "data/Android/Processed/Stan-Desk/files/android_build-Pixel_3-1564483096188.csv"

In [24]:
df = read_csv(data_file)
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3
0,1564483096327,ACC,0,0,0,493750930.0,-120897930.0,9.837141e+09
1,1564483096329,ACC,0,0,0,465154680.0,-130527000.0,9.908659e+09
2,1564483096330,ACC,0,0,0,431792440.0,-116083440.0,9.846677e+09
3,1564483096331,ACC,0,0,0,479451780.0,-130527000.0,9.837141e+09
4,1564483096332,ACC,0,0,0,493749260.0,-149785190.0,9.846677e+09


In [7]:
df = read_csv("/Users/stanzhang/Developer/Brave/Analysis/Jupyter/data/Bot/data/Stan-Swing/android_build-Pixel_3-1565690457215.csv")
# df = read_csv("/Users/stanzhang/Developer/Brave/Analysis/Jupyter/data/Android/Processed/Stan-Handheld/files/android_build-Pixel_3-1564481961365.csv")


In [8]:
segs = get_click_segs(df, '5ms')

In [16]:
segs[0][0]

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,amp
timestamp,,,,,,,,,
2019-08-13 10:00:57.940,1565690457940,UAC,0.0,0.0,0.0,-23152.0,359743.0,-41111.0,3.628239e+05
2019-08-13 10:00:57.945,1565690457945,UAC,0.0,0.0,0.0,-23152.0,359743.0,-41111.0,3.628239e+05
2019-08-13 10:00:57.950,1565690457950,UAC,0.0,0.0,0.0,10693.0,301064.0,209984.0,3.672154e+05
2019-08-13 10:00:57.955,1565690457955,UAC,0.0,0.0,0.0,10693.0,301064.0,209984.0,3.672154e+05
2019-08-13 10:00:57.960,1565690457960,UAC,0.0,0.0,0.0,10693.0,301064.0,209984.0,3.672154e+05
2019-08-13 10:00:57.965,1565690457965,UAC,0.0,0.0,0.0,-50426.0,-52505.0,-4324.0,7.292636e+04
2019-08-13 10:00:57.970,1565690457970,UAC,0.0,0.0,0.0,-50426.0,-52505.0,-4324.0,7.292636e+04
2019-08-13 10:00:57.975,1565690457975,UAC,0.0,0.0,0.0,325046.0,307344.0,-702633.0,8.329516e+05
2019-08-13 10:00:57.980,1565690457980,UAC,0.0,0.0,0.0,325046.0,307344.0,-702633.0,8.329516e+05


In [73]:
segs = get_click_segs(df)

In [58]:
seg = segs[0][0]

seg = seg[seg['sensor'] == 'GYR']
seg_z = seg['d2']
seg_time = seg['timestamp']

z_diff = seg_z[1:].values - seg_z[:-1].values
time_diff = seg_time[1:].values - seg_time[:-1].values

dev = z_diff / time_diff 

plt.figure()
plt.plot(range(dev.size), dev)
plt.show()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [154]:
dev[np.argmin(dev)]

-63774332.0

In [84]:
plot_for_slides(segs[2][0])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
plot_for_slides(segs[2][0], dev=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:

df = read_csv("data/Android/Processed/Stan-Handheld/files/android_build-Pixel_3-1564481961365.csv")

segs = get_click_segs(df, '5ms')
plot_for_slides(segs[3][0])


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
df = read_csv("data/Android/Processed/Stan-Handheld/files/android_build-Pixel_3-1564481961365.csv")

segs = get_click_segs(df, '5ms')
plot_for_slides(segs[3][0], dev=True)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [90]:
plot_sensor_axis('GYR', 'y', segs[5][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
cs = segs[1][0]

css = cs[cs['sensor'] == 'GYR']

css1 = css[css['timestamp'] <= css[css['touch'] == 1].iloc[-1, 0] + TIME_BEFORE]
css2 = css[css['timestamp'] > css[css['touch'] == 1].iloc[-1, 0] + TIME_BEFORE]


css2

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,amp
timestamp,,,,,,,,,
2019-07-30 10:38:20.128,1564483100128,GYR,0.0,0.0,0.0,-0.000020,-0.000945,-0.000602,0.001121
2019-07-30 10:38:20.132,1564483100132,GYR,0.0,0.0,0.0,-0.000020,-0.000945,-0.000602,0.001121
2019-07-30 10:38:20.136,1564483100136,GYR,0.0,0.0,0.0,-0.000020,-0.000945,-0.000602,0.001121
2019-07-30 10:38:20.140,1564483100140,GYR,0.0,0.0,0.0,-0.000020,-0.000945,-0.000602,0.001121
2019-07-30 10:38:20.144,1564483100144,GYR,0.0,0.0,0.0,-0.000020,-0.003076,-0.000602,0.003135
2019-07-30 10:38:20.148,1564483100148,GYR,0.0,0.0,0.0,0.002111,-0.002012,0.000464,0.002953
2019-07-30 10:38:20.152,1564483100152,GYR,0.0,0.0,0.0,0.002111,-0.002012,0.000464,0.002953
2019-07-30 10:38:20.156,1564483100156,GYR,0.0,0.0,0.0,0.002111,-0.002012,0.000464,0.002953
2019-07-30 10:38:20.160,1564483100160,GYR,0.0,0.0,0.0,0.002111,-0.002012,0.000464,0.002953


In [ ]:
c

In [155]:
cs.index.values.astype(np.int64) // 10 ** 6

array([1564483098824, 1564483098828, 1564483098832, 1564483098836,
       1564483098840, 1564483098844, 1564483098848, 1564483098852,
       1564483098856, 1564483098860, 1564483098864, 1564483098868,
       1564483098872, 1564483098876, 1564483098880, 1564483098884,
       1564483098888, 1564483098892, 1564483098896, 1564483098900,
       1564483098904, 1564483098908, 1564483098912, 1564483098916,
       1564483098920, 1564483098924, 1564483098928, 1564483098932,
       1564483098936, 1564483098940, 1564483098944, 1564483098948,
       1564483098952, 1564483098956, 1564483098960, 1564483098964,
       1564483098968, 1564483098972, 1564483098976, 1564483098980,
       1564483098984, 1564483098988, 1564483098992, 1564483098996,
       1564483099000, 1564483099004, 1564483099008, 1564483099012,
       1564483099016, 1564483099020, 1564483099024, 1564483099028,
       1564483099032, 1564483099036, 1564483099040, 1564483099044,
       1564483099048, 1564483099052, 1564483099056, 1564483099

In [148]:
css[css['touch'] == 1].iloc[-1]

timestamp      1.56448e+12
sensor                 UAC
touch                    1
keyboard                 0
orientation              0
d1             0.000956044
d2               0.0107443
d3               0.0160179
amp              0.0193114
Name: 2019-07-30 10:38:18.920000, dtype: object

In [146]:
css1

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,amp
timestamp,,,,,,,,,
2019-07-30 10:38:18.824,1.564483e+12,UAC,0.0,0.0,0.0,0.001603,0.010546,0.054159,0.055200
2019-07-30 10:38:18.828,1.564483e+12,UAC,0.0,0.0,0.0,0.001603,0.010546,0.054159,0.055200
2019-07-30 10:38:18.832,1.564483e+12,UAC,0.0,0.0,0.0,0.001603,0.010546,0.054159,0.055200
2019-07-30 10:38:18.836,1.564483e+12,UAC,0.0,0.0,0.0,0.001603,0.010546,0.054159,0.055200
2019-07-30 10:38:18.840,1.564483e+12,UAC,0.0,0.0,0.0,0.025480,-0.013595,0.101839,0.105855
2019-07-30 10:38:18.844,1.564483e+12,UAC,0.0,0.0,0.0,0.025480,-0.013595,0.101839,0.105855
2019-07-30 10:38:18.848,1.564483e+12,UAC,0.0,0.0,0.0,0.025480,-0.013595,0.101839,0.105855
2019-07-30 10:38:18.852,1.564483e+12,UAC,0.0,0.0,0.0,0.016162,0.010507,0.025549,0.032006
2019-07-30 10:38:18.856,1.564483e+12,UAC,0.0,0.0,0.0,0.016162,0.010507,0.025549,0.032006


In [135]:
extract_features(segs, 'label', 'remark')

,uac_min_x,uac_min_y,uac_min_z,uac_min_amp,uac_max_x,uac_max_y,uac_max_z,uac_max_amp,uac_gap_x,uac_gap_y,...,touch_x,touch_y,touch_duration,click_duration,uac_num1,uac_num2,gyr_num1,gyr_num2,label,remark
0,-0.044751,-0.037520,-1.309468,0.016963,0.025574,0.025115,0.168574,1.310470,36.0,21.0,...,99,1966,52,299.0,62,14,62,14,label,remark
1,-0.025421,-0.020232,-0.055684,0.035792,0.058381,0.051958,0.654717,0.657345,37.0,15.0,...,390,713,122,415.0,60,44,60,45,label,remark
2,-0.028373,-0.001254,-0.084596,0.010065,0.053041,0.064885,0.072753,0.105602,90.0,80.0,...,67,1997,64,353.0,53,37,52,38,label,remark
3,-0.032735,-0.016207,-1.171706,0.010324,0.038829,0.012439,0.034579,1.171944,62.0,14.0,...,424,948,29,322.0,33,48,33,48,label,remark
4,-0.065794,-0.034614,-0.533002,0.028186,0.039670,0.032727,0.053456,0.538044,22.0,22.0,...,108,1962,53,339.0,43,42,43,43,label,remark
5,-0.055548,-0.021411,-0.623604,0.013232,0.034649,0.041633,0.039144,0.626172,37.0,86.0,...,653,723,38,325.0,34,48,35,49,label,remark
6,-0.022988,-0.056925,-0.071105,0.021345,0.048786,0.024655,0.024266,0.086777,49.0,24.0,...,68,1977,53,342.0,53,31,54,33,label,remark
7,-0.015236,-0.033477,-0.095023,0.034478,0.042297,0.072460,0.114772,0.120720,12.0,31.0,...,912,738,53,317.0,35,43,35,45,label,remark
8,-0.060527,-0.044919,-0.057130,0.012911,0.048297,0.008505,0.371991,0.375698,21.0,37.0,...,66,1988,37,321.0,37,44,40,44,label,remark
9,-0.018628,-0.015715,-0.052452,0.040525,0.033935,0.046978,0.123952,0.127829,15.0,22.0,...,675,948,29,310.0,30,48,33,49,label,remark


In [130]:
plot_sensor_axis('GYR', 'x', segs[0][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
segs[0][0].shape

(152, 9)

In [29]:
seg = segs[0][0]

plt.figure()
d1 = seg[1:]['d1'].values - seg[:-1]['d1'].values
plt.plot(seg[1:]['timestamp'], d1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
seg = segs[1][0]
sensor = 'UAC'
axis = 'z'

sseg = seg[seg['sensor'] == sensor]

d1 = sseg[1:][AXIS_INDEX[axis]].values - sseg[:-1][AXIS_INDEX[axis]].values
d2 = sseg[1:]['timestamp'].values - sseg[:-1]['timestamp'].values
d2 = np.where(d2==0, 0.5, d2) 
d3 = d1 / d2

seg2 = sseg[1:]
seg2[AXIS_INDEX[axis]] = d3
seg2['timestamp'] = range(len(d3))

plot_sensor_axis(sensor, axis, seg2)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [250]:
ddf = downsampling(segs[0][0], 5)

In [254]:
plot_sensor_axis("UAC", "z", ddf)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [255]:
plot_sensor_axis("UAC", "z", segs[0][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [164]:
df['touch'].value_counts().get(1, 0)

42047

In [108]:
variation(df['d1'])

13.184634809286077

In [98]:
segs = get_drag_segs(df)

In [99]:
plot_sensor_axis("GYR", "y", segs[4][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

In [57]:
d1 = df['d1']

In [58]:
d1[4]

0.49374926

In [61]:
df['d1'].values[4]

0.49374926

In [26]:
df[df['sensor'] == 'GYR'].head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
16,1564483096346,GYR,0,0,0,0.002031,0.002306,-0.001613,-0.000861,0.002614,-0.00081
17,1564483096347,GYR,0,0,0,-0.003296,0.002306,-0.003744,-0.000861,0.002614,-0.00081
18,1564483096347,GYR,0,0,0,-0.004361,0.004436,0.001583,-0.000861,0.002614,-0.00081
19,1564483096348,GYR,0,0,0,-0.000100,0.000175,0.001583,-0.000861,0.002614,-0.00081
20,1564483096349,GYR,0,0,0,-0.001165,0.000175,-0.000548,-0.000861,0.002614,-0.00081


In [27]:
df[['d1', 'd2', 'd3']] = df[df['sensor'] == 'GYR'][['d1', 'd2', 'd3']] - df[df['sensor'] == 'GYR'][['d4', 'd5', 'd6']].values

In [28]:
df[df['sensor'] == 'GYR'].head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
16,1564483096346,GYR,0,0,0,0.002892,-0.000309,-0.000803,-0.000861,0.002614,-0.00081
17,1564483096347,GYR,0,0,0,-0.002434,-0.000309,-0.002934,-0.000861,0.002614,-0.00081
18,1564483096347,GYR,0,0,0,-0.003500,0.001822,0.002392,-0.000861,0.002614,-0.00081
19,1564483096348,GYR,0,0,0,0.000761,-0.002439,0.002392,-0.000861,0.002614,-0.00081
20,1564483096349,GYR,0,0,0,-0.000304,-0.002439,0.000262,-0.000861,0.002614,-0.00081


In [16]:
df[df['sensor'] == 'GYR'].head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
16,1564483096346,GYR,0,0,0,0.002031,0.002306,-0.001613,-0.000861,0.002614,-0.00081
17,1564483096347,GYR,0,0,0,-0.003296,0.002306,-0.003744,-0.000861,0.002614,-0.00081
18,1564483096347,GYR,0,0,0,-0.004361,0.004436,0.001583,-0.000861,0.002614,-0.00081
19,1564483096348,GYR,0,0,0,-0.000100,0.000175,0.001583,-0.000861,0.002614,-0.00081
20,1564483096349,GYR,0,0,0,-0.001165,0.000175,-0.000548,-0.000861,0.002614,-0.00081


In [182]:
click_segs = get_click_segs(df)

In [198]:
click_segs[i][0][click_segs[i][0]['sensor']=='ACC']

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6,amp
1,1564483450423,ACC,0,0,0,0.432121,-0.216389,9.759778,0.374047,-0.296235,0.011822,9.771736
3,1564483450423,ACC,0,0,0,0.475019,-0.173058,9.826529,0.374047,-0.296235,0.011822,9.839526
5,1564483450423,ACC,0,0,0,0.465491,-0.168243,9.845599,0.374047,-0.296235,0.011822,9.858033
9,1564483450424,ACC,0,0,0,0.432134,-0.182687,9.764546,0.374047,-0.296235,0.011822,9.775811
10,1564483450424,ACC,0,0,0,0.436904,-0.163428,9.764546,0.374047,-0.296235,0.011822,9.775682
13,1564483450425,ACC,0,0,0,0.465504,-0.168242,9.821760,0.374047,-0.296235,0.011822,9.834224
14,1564483450425,ACC,0,0,0,0.446444,-0.158613,9.836064,0.374047,-0.296235,0.011822,9.847468
15,1564483450425,ACC,0,0,0,0.394022,-0.163427,9.840831,0.374047,-0.296235,0.011822,9.850072
19,1564483450425,ACC,0,0,0,0.436920,-0.197129,9.878974,0.374047,-0.296235,0.011822,9.890596
21,1564483450426,ACC,0,0,0,0.432159,-0.182685,9.869438,0.374047,-0.296235,0.011822,9.880584


In [199]:
for i in range(len(click_segs)):
    print(len(click_segs[i][0][click_segs[i][0]['sensor']=='ACC']))

123
169
146
132
142
136
140
150
135
132
141
152
137
148
143
139
161
135
137
181
150
161
146
143
147
168
156
176
162
151
141
117
143
144
141
138
136
140
140
137
136
138
138
137
133
148
137
134
134
137
135
141
148
140
134
137
147
138
142
139
136
141
135
137
137
136
139
148
123
133
132
140
134
139
138
143
134
138
143
144
140
131
143
152
139
140
151
136
139
142
139
143
140
140
136
143
139
139
138
136
132
138
137
151
147
134
137
131
139
139
140
138
130
139
133
147
142
132
135
140
140
141
137
137
141
133
125
125
132
131
137
138
140
135
140
137
139
141
134
139
138
133
138
141
138
141
142
134
138
135
143
140
145
143
140
140
144
137
141
143
136
138
134
131
139
138
140
139
141
143
144
138
125
140
135
149


In [176]:
extract_features(click_segs, 'Human', 'Stan-Desk')

,uac_min_x,uac_min_y,uac_min_z,uac_max_x,uac_max_y,uac_max_z,uac_gap_x,uac_gap_y,uac_gap_z,uac_std1_x,...,gyr_std1_y,gyr_std1_z,gyr_std2_x,gyr_std2_y,gyr_std2_z,touch_x,touch_y,touch_duration,label,remark
0,-0.044751,-0.037520,-1.309468,0.025574,0.025115,0.168574,36,21,15,0.026169,...,0.009829,0.001550,0.001587,0.001460,0.001362,99,1966,52,Human,Stan-Desk
1,-0.025421,-0.020232,-0.055684,0.058381,0.056403,0.654717,37,137,13,0.022305,...,0.015386,0.001125,0.001828,0.001963,0.001605,390,713,122,Human,Stan-Desk
2,-0.028373,-0.001254,-0.084596,0.053041,0.064885,0.072753,90,80,29,0.029739,...,0.003340,0.001432,0.002105,0.002083,0.001583,67,1997,64,Human,Stan-Desk
3,-0.032735,-0.016207,-1.171706,0.038829,0.012439,0.034579,62,14,14,0.019870,...,0.010787,0.001580,0.001463,0.001770,0.001495,424,948,29,Human,Stan-Desk
4,-0.065794,-0.034614,-0.533002,0.039670,0.032727,0.053456,22,22,15,0.036122,...,0.006506,0.001462,0.001660,0.001784,0.001661,108,1962,53,Human,Stan-Desk
5,-0.055548,-0.021411,-0.623604,0.034649,0.041633,0.039144,37,86,50,0.025774,...,0.004339,0.001798,0.001557,0.002139,0.001857,653,723,38,Human,Stan-Desk
6,-0.022988,-0.056925,-0.071105,0.048786,0.024655,0.024266,49,24,43,0.022008,...,0.005369,0.001691,0.001753,0.001930,0.001488,68,1977,53,Human,Stan-Desk
7,-0.015236,-0.033477,-0.095023,0.042297,0.072460,0.114772,12,31,48,0.019716,...,0.002890,0.001450,0.001649,0.001623,0.001532,912,738,53,Human,Stan-Desk
8,-0.060527,-0.044919,-0.057130,0.048297,0.008505,0.371991,21,37,21,0.032428,...,0.008150,0.001454,0.001537,0.002157,0.001433,66,1988,37,Human,Stan-Desk
9,-0.018628,-0.015715,-0.052452,0.043214,0.046978,0.123952,41,22,27,0.019514,...,0.003074,0.001366,0.001814,0.001937,0.001490,675,948,29,Human,Stan-Desk


In [128]:
plot_sensor_axis("UAC", 'z', click_segs[3][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

...

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

### Handheld

In [123]:
data_file = "data/Android/Processed/Stan-Handheld/files/android_build-Pixel_3-1564481961365.csv"


In [124]:
df = read_csv(data_file)
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
0,1564481961511,ACC,0,0,0,-0.623700,6.376127,7.789820,0.374047,-0.296235,0.011822
1,1564481961513,ACC,0,0,0,-0.599872,6.390571,7.847035,0.374047,-0.296235,0.011822
2,1564481961513,ACC,0,0,0,-0.604640,6.405014,7.885177,0.374047,-0.296235,0.011822
3,1564481961515,ACC,0,0,0,-0.576046,6.433901,7.937624,0.374047,-0.296235,0.011822
4,1564481961516,ACC,0,0,0,-0.571283,6.453159,7.994838,0.374047,-0.296235,0.011822


In [125]:
click_segs = get_click_segs(df)

In [129]:
plot_sensor_axis("UAC", 'z', click_segs[2][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

RuntimeError: libpng signaled error

### Little Stand

In [131]:
data_file = "data/Android/Processed/Stan-LittleStand/files/android_build-Pixel_3-1564483809686.csv"


In [132]:
df = read_csv(data_file)
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
0,1564483809807,ACC,0,0,1,3.949314,-0.337756,9.004119,0.374047,-0.296235,0.011822
1,1564483809809,ACC,0,0,1,4.006508,-0.352199,9.113780,0.374047,-0.296235,0.011822
2,1564483809809,ACC,0,0,1,3.973148,-0.260723,9.166226,0.374047,-0.296235,0.011822
3,1564483809810,ACC,0,0,1,3.906426,-0.313683,9.123316,0.374047,-0.296235,0.011822
4,1564483809811,ACC,0,0,1,3.949322,-0.352199,9.018422,0.374047,-0.296235,0.011822


In [133]:
click_segs = get_click_segs(df)

In [134]:
plot_sensor_axis("UAC", 'z', click_segs[2][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

RuntimeError: libpng signaled error

## OnePlus (Inigo)

In [34]:
data_file = "/Users/stanzhang/Developer/Brave/Analysis/Jupyter/data/Android/Processed/Inigo/files/OnePlus-ONEPLUS_A6003-1562339146716.csv"

In [35]:
df = read_csv(data_file)

In [40]:
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
0,1562339146861,ACC,0,0,0,-0.154313,-0.054486,9.665712,0.0,0.0,0.0
1,1562339146861,ACC,0,0,0,-0.161499,-0.066463,9.900470,0.0,0.0,0.0
2,1562339146862,ACC,0,0,0,-0.161499,-0.076045,9.950774,0.0,0.0,0.0
3,1562339146862,ACC,0,0,0,-0.130358,-0.023345,9.943588,0.0,0.0,0.0
4,1562339146862,ACC,0,0,0,-0.111194,-0.054486,10.001080,0.0,0.0,0.0


In [37]:
click_segs = get_click_segs(df)

In [38]:
extract_features(click_segs, 'Human', 'Inigo')

,uac_min_x,uac_min_y,uac_min_z,uac_min_amp,uac_max_x,uac_max_y,uac_max_z,uac_max_amp,uac_gap_x,uac_gap_y,...,gyr_std1_amp,gyr_std2_x,gyr_std2_y,gyr_std2_z,gyr_std2_amp,touch_x,touch_y,touch_duration,label,remark
0,-0.050406,-0.041714,-0.224852,0.014451,0.111709,0.055481,0.227991,0.255403,4,86,...,0.020105,0.001749,0.00182,0.000998,0.001624,289,152,163,Human,Inigo
1,-0.835671,-2.309898,-2.423070,0.502916,1.337241,0.981701,1.372467,2.989607,54,48,...,0.676396,0.096563,0.22725,0.059334,0.234565,334,1551,99,Human,Inigo


In [44]:
seg = click_segs[0][0]

In [47]:
for axis in ['x', 'y', 'z']:
    plot_sensor_axis('GYR', axis, seg)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
sensor = "UAC"

click_segs = get_click_segs(df, sensor)
seg = click_segs[1][0]

for axis in ['x', 'y', 'z']:
    plot_sensor_axis(sensor, axis, seg)

TypeError: get_click_segs() takes 1 positional argument but 2 were given

In [83]:
sensor = "UAC"

# for axis in ['x', 'y', 'z']:
#     plot_sensor_axis(sensor, axis, df)

plot_sensor_amplitude(sensor, df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
sensor = "GRA"

click_segs = get_click_segs(df)
seg = click_segs[1][0]

for axis in ['x', 'y', 'z']:
    plot_sensor_axis(sensor, axis, seg)

IndexError: single positional indexer is out-of-bounds

In [25]:
fft = np.fft.fft(seg['d3'])

plt.figure()
plt.plot(fft)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/local/lib/python3.7/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [38]:
fft = np.fft.fft(seg['d3'])
ifft = np.fft.ifft(fft)

plt.figure()
plt.plot(ifft)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/usr/local/lib/python3.7/site-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [40]:
plot_sensor_axis("UAC", "z", click_segs[0][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
print("timestamp, is_touch, x, y, z")
      
for i in range(len(click_segs[0][0])):
    print("{}, {}, {:.9f}, {:.9f}, {:.9f}".format(click_segs[0][0].iloc[i, 0], click_segs[0][0].iloc[i, 2], click_segs[0][0].iloc[i, 5], click_segs[0][0].iloc[i, 6], click_segs[0][0].iloc[i, 7]))

timestamp, is_touch, x, y, z
1562339164009, 0, -0.010682195, 0.033716172, 0.013777733
1562339164013, 0, 0.016736746, 0.027531294, 0.005348206
1562339164021, 0, 0.007157892, 0.014373427, 0.036187172
1562339164023, 0, 0.014236197, 0.043930380, 0.032321930
1562339164028, 0, 0.016375930, 0.011528647, -0.004770279
1562339164035, 0, 0.030401080, 0.026881158, 0.045154570
1562339164039, 0, -0.002449617, 0.055480756, -0.224851610
1562339164043, 0, -0.050056085, 0.009680115, -0.107835770
1562339164048, 0, 0.111708700, 0.027783414, 0.227991100
1562339164052, 0, -0.050405815, 0.050383884, -0.177759170
1562339164057, 1, 0.008245528, 0.023296352, 0.071914670
1562339164062, 1, 0.030795530, 0.023393330, 0.146114350
1562339164067, 1, -0.027848884, 0.015072988, 0.065299034
1562339164072, 1, 0.021021709, 0.016450847, 0.011289597
1562339164077, 1, -0.003847614, 0.017638069, 0.037312508
1562339164082, 1, 0.003688194, 0.035495725, 0.004922867
1562339164086, 1, 0.012580067, 0.009411318, -0.008191109
15623391

In [209]:
plot_sensor_axis("ACC", "z", df=click_segs[0][0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [230]:
extract_features(click_segs)

,min_x,min_y,min_z,max_x,max_y,max_z,diff_x,diff_y,diff_z,gap_x,...,std1_x,std1_y,std1_z,std2_x,std2_y,std2_z,touch_x,touch_y,touch_duration,label
0,-0.050386,-0.041446,-0.070043,0.039505,0.098119,0.087184,0.089891,0.139565,0.157228,87,...,0.028607,0.040087,0.040786,0.034529,0.026589,0.033023,730,1806,96,click
1,-0.043292,-0.045804,-0.018382,0.065939,0.045892,0.062361,0.109231,0.091696,0.080742,12,...,0.030778,0.026999,0.025752,0.021050,0.039378,0.039721,719,1784,78,click
2,-0.083619,-0.074270,-0.061809,0.031471,0.051025,0.095528,0.115090,0.125296,0.157337,124,...,0.030424,0.033241,0.043332,0.024901,0.042713,0.042303,557,1776,84,click
3,-0.043639,-0.002151,-0.042424,0.041950,0.055557,0.081231,0.085589,0.057709,0.123655,25,...,0.025057,0.016949,0.035388,0.026293,0.023593,0.035666,476,1800,77,click
4,-0.039684,-0.055738,-0.051239,0.055691,0.122222,0.120600,0.095375,0.177960,0.171839,49,...,0.030131,0.046154,0.049527,0.026619,0.029392,0.035931,366,1657,92,click
5,-0.020711,-0.061209,-0.059315,0.061056,0.160249,0.073587,0.081767,0.221457,0.132902,74,...,0.022498,0.052778,0.051098,0.038901,0.068378,0.048650,590,1636,67,click
6,-0.070766,-0.055309,-0.070628,0.067288,0.141909,0.119997,0.138055,0.197218,0.190625,38,...,0.041382,0.056283,0.050514,0.048547,0.063069,0.060256,385,1757,84,click
7,-0.048637,-0.018725,-0.070457,0.094109,0.082259,0.101305,0.142746,0.100985,0.171762,38,...,0.035004,0.025108,0.054613,0.028166,0.047567,0.044273,638,1246,69,click
8,-0.315688,-0.515456,-0.383704,0.123549,0.628193,0.714954,0.439238,1.143650,1.098658,26,...,0.113525,0.281127,0.313516,0.031209,0.061316,0.044525,796,917,46,click
9,-0.906719,-0.172017,-0.870425,1.101075,0.531305,0.356155,2.007794,0.703322,1.226580,12,...,0.413341,0.162167,0.285422,0.028072,0.047469,0.046137,829,1403,62,click


In [60]:
plot_sensor_axis("UAC", "z", df=cs, ptype='scatter')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Accelerometer outputs

In [85]:
plot_sensor_axis("ACC", "x", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
plot_sensor_output("ACC", "y", df=df)

NameError: name 'plot_sensor_output' is not defined

In [169]:
plot_sensor_axis("ACC", "z", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
plot_sensor_amplitude("ACC", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Gyroscope Outputs

In [77]:
plot_sensor_axis("GYR", "x", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plot_sensor_axis("GYR", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
plot_sensor_axis("GYR", "z", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plot_sensor_amplitude("GYR", df=df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### User Acceleration

In [30]:
plot_sensor_output("UAC", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
plot_sensor_output("UAC", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plot_sensor_output("UAC", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Rotation

In [18]:
plot_sensor_output("ROT", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plot_sensor_output("ROT", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plot_sensor_output("ROT", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Gravity

In [21]:
plot_sensor_output("GRA", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
plot_sensor_output("GRA", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plot_sensor_output("GRA", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Samsung (Alex)

In [261]:
data_file = "/Users/stanzhang/Developer/Brave/Analysis/Jupyter/data/Android/Processed/Alex/files/dpi-SM_G920F-1563894180164.csv"

In [262]:
df = read_csv(data_file)

In [263]:
df.head()

,timestamp,sensor,touch,keyboard,orientation,d1,d2,d3,d4,d5,d6
0,1563894180276,ROT,0,0,0,0.163029,0.038995,0.000000,0.985850,246.0,NaN
1,1563894180277,UAC,0,0,0,0.056089,-0.061377,-0.223410,NaN,NaN,NaN
2,1563894180277,ROT,0,0,0,0.163029,0.038995,0.000000,0.985850,246.0,NaN
3,1563894180277,UAC,0,0,0,0.094396,-0.066165,-0.187496,NaN,NaN,NaN
4,1563894180277,ROT,0,0,0,0.162967,0.034946,0.000667,0.986012,246.0,NaN


In [267]:
cs = get_click_segs(df)

In [268]:
len(cs)

26

## User Acceleration outputs

In [58]:
plot_sensor_axis("UAC", "x", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [159]:
plot_sensor_axis("UAC", "y", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [264]:
plot_sensor_axis("UAC", "z", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [156]:
plot_sensor_amplitude("UAC", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gyroscope Outputs

In [77]:
plot_sensor_axis("GYR", "x", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
plot_sensor_axis("GYR", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
plot_sensor_axis("GYR", "z", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plot_sensor_amplitude("GYR", df=df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Google Pixel 3 (Stan)

In [92]:
data_file = "data/Android/android_build-Pixel 3-1563801044184-fix.csv"

In [93]:
df = pd.read_csv(data_file, header=None)

In [94]:
df = df.reset_index(drop=True)

## Accelerometer outputs

In [21]:
plot_sensor_axis("ACC", "x", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plot_sensor_axis("ACC", "y", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [96]:
plot_sensor_axis("ACC", "z", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [95]:
plot_sensor_amplitude("ACC", df=df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gyroscope Outputs

In [17]:
plot_sensor_output("GYR", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plot_sensor_output("GYR", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
plot_sensor_output("GYR", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## User Acceleration

In [20]:
plot_sensor_output("UAC", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
plot_sensor_output("UAC", "y", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
plot_sensor_output("UAC", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Rotation

In [22]:
plot_sensor_output("ROT", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plot_sensor_output("ROT", "y", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
plot_sensor_output("ROT", "z", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Gravity

In [23]:
plot_sensor_output("GRA", "x", df = df)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
plot_sensor_output("GRA", "y", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
plot_sensor_output("GRA", "z", df = df)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …